In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from google.colab import files
from google.colab import drive
import numpy as np
drive.mount('/content/drive')
example1 = '/content/drive/MyDrive/eminemCL.txt'
text = open(example1, 'r', encoding='utf-8').read()
# turn text to lower case to reduce vocabulary
text = text.lower()
with open(example1) as f:
    content = f.readlines()
# Defined content in case we need to split each line/bar into strings
content = [x.strip() for x in content] 
print(content)
Vocabulary = ''.join([i for i in text if not i.isdigit()]).replace("\n"," ").split(' ')
print(Vocabulary)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['ook, I was gonna go easy on you and not to hurt your feelings', "But I'm only going to get this one chance", "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)", "Just a feeling I've got, like something's about to happen, but I don't know what", "If that means, what I think it means, we're in trouble, big trouble,", "And if he is as bananas as you say, I'm not taking any chances", "You were just what the doctor ordered I'm beginning to feel like a Rap God, Rap God", 'All my people from the front to the back nod, back nod', 'Now who thinks their arms are long enough to slap box, slap box?', 'They said I rap like a robot, so call me Rapbot But for me to rap like a computer must be in my genes', 'I got a laptop in my back pocket', "My pen'll go off when I half-penis it", 'Got a fat knot from that rap profit', 'Made a living and a killing of

In [ ]:

#@title Function Definitions
# Recursively Import the Data (AUTOMATIC)

def _decode_and_write(file__, path_):
    data = file__.decoded_content
    data = data.decode('utf-8')[1:]
    with open(path_, 'w') as writefile:
        writefile.write(data) 
    data = data.splitlines()
    data_rows = []
    for count, word in enumerate(data):
        if count>0:
            data_rows.append(word.split(','))
    data = pd.DataFrame(data_rows)
    data = data.to_numpy()
    return data


def import_github(path_name="AllLyrics.txt"):
    """
    Function for importing the github file
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()
    print("Importing Github cleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    RAP_DATA = []
    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the Lyrics
            RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
    
    temp_path = Path(path_name)
    if temp_path.is_file(): 
        if os.stat(path_name).st_size == 0:
            write_bool2 = True
        else: 
            write_bool2 = False
    else: 
        write_bool2 = True
    
    if write_bool2: 
        for lyric in RAP_DATA: 
            try:
                with open(path_name, 'w') as writefile: 
                    writefile.write(lyric)
            except: 
                print("Error, file moved/deleted during write")
        print("{} is now up to date!".format(path_name))
    else: 
        print("{} is already up to date!".format(path_name))
    
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 

        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        path = path[title_start + 6:title_start + title_len + 4]

        print("Writing file {} {}".format(counter, path))
        temp_path = Path(path)
        if temp_path.is_file():
            with open(path,'w'): pass # Cheeky way to clear the file if it exists
        
        # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy array 
        data = file_.decoded_content
        data = data.decode('utf-8')[1:]

        with open(path, 'w') as writefile:
            writefile.write(data) 
        print("All files now up to date!")


def update_github(write_bool=False, path_name="AllLyrics.txt"):
    """
    Function for updating the github file, by cleaning the lyrics, optional write to txt file. 
    write_bool: bool
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()

    print("Importing editing csv files...")

    # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy arr
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path 
        path = str(path)
        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len + 4]
        print("Writing file {} {}".format(counter, name))
        if name.lower() == "censors.csv":
            censors = _decode_and_write(file_, path)
        elif name.lower() == "capitals.csv":
            capitals = _decode_and_write(file_, path)
        else: 
            _decode_and_write(file_, path)
    print("All editing csv files are up to date!")

    print("Importing Github uncleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/UNCLEAN", ref=branch)

    RAP_DATA = []
    rap_lyric_names = []

    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('UNCLEAN')
            title_len = path[title_start:].find('.')
            name = path[title_start + 8:title_start + title_len]
            if name[-2:] == 'UC':
                name = name[:-2]
            rap_lyric_names.append(name) 

        # Append the Lyrics
        RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
        
    # Remove the \ufeff at the beginning O(n)
    for count, lyric in enumerate(RAP_DATA): 
        RAP_DATA[count] = lyric[1:]

    # Censor the profanities O(n*m + n*m2) m > m2 xor m2 > m
    for count in range(len(RAP_DATA)): 
        for i in range(len(censors[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(censors[i, 0]), str(censors[i, 1]))
        for i in range(len(capitals[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(capitals[i, 0]), str(capitals[i, 1]))

    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    cleaned_names = []
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 
        print("File {} ".format(counter + 1) + path)
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('CLEAN')
            title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len]
        if name[-2:] == 'CL':
            name = name[:-2]
        cleaned_names.append(name) 

    # ALL OF THE EDITING IS DONE IN THE 'PROTOTYPE BRANCH' to avoid overwriting import changes
    # If the (now cleaned) rap_lyrics name is new (not in cleaned_names), then we want to create that as a new file 
    # If the (now cleaned) rap_lyrics name is NOT new (not in cleaned_names), then we want to update the file
    # print(rap_lyric_names)
    # print(cleaned_names)
    print("Commiting files to github...")
    for counter, new_name in enumerate(rap_lyric_names): 
        if new_name in cleaned_names: 
            duplicate = r_proj_clone.get_contents("RapLyrics/CLEAN/{}CL.txt".format(new_name), ref=branch)
            r_proj_clone.update_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], duplicate.sha, branch=branch)
        else:
            r_proj_clone.create_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], branch=branch)

    if write_bool: 
        print("Writing text file to: {}".format(path_name))
        with open(path_name, 'w') as writefile:
            for lyric in RAP_DATA:
                writefile.write(lyric)

In [ ]:
words = sorted(list(set(Vocabulary)))
print(words)
intdict = { i : words[i] for i in range(len(words))}
print(intdict)
charsdict = { words[i] : i for i in range(len(words))}
print(charsdict)

['', '"', '"ahh', '"all', '"baby,', '"becky', '"bitch!"', '"but', '"cut', '"dad\'s', '"daddy', '"daddy,', '"dear', '"debbie', '"did', '"don\'t', '"dre,', '"evol"', '"frick', '"fuck', '"get', '"guilty', '"hell', '"hey,', '"i', '"i\'ll', '"i\'m', '"in', '"infinite"', '"kid")', '"let\'s', '"marshall', '"my', '"nicki,', '"nightmare', '"ninjas', '"no."', '"not', '"oh', '"oh,', '"please', '"poot', '"pubic', '"relapse"', '"shady', '"slim', '"so', '"sup', '"testing', '"the', '"this', '"touch', '"wait!",', '"we', '"well', '"whore",', '"yeah', '"yeah,', '"you', '"you\'re', '$.', '&', "'", '\'"]', "''hi!", "',", "'baby", "'back", "'bout", "'burbs", "'cause", "'cause,", "'causing", "'cid", "'cleaning", "'cos", "'cus", "'cuz", "'dicted", "'em", "'em,", "'em?", "'er", "'ey", "'fore", "'freeze!':", "'freeze'?", "'ice,", "'matic,", "'n'", "'pee-pee'", "'round", "'s", "'shrooms", "'the", "'til", "'till", "'why,", "'′", "'″", '(', "('cause", '(a', '(ah!)', '(ah-ha', '(ahh!)', '(and', '(argh!)', '(ask', 

In [ ]:
print(len(words))
print(len(text))

7746
323318


In [ ]:
seq_len = 20
examples_per_epoch = len(Vocabulary)//(seq_len + 1)

step = 1 
sentences = [] 
nextword = [] 

# We need training and test examples so we split our text into chunks of 'seq_len' with the next word in the list being our 'output'
for i in range(0, len(Vocabulary) - seq_len, step):
    train_set  = Vocabulary[i:i+seq_len]
    print(train_set)
    # Output sequence (will be used as target)
    test_set = Vocabulary[i + seq_len]
    print(test_set)
    # Store samples in data_X
    sentences.append([charsdict[char] for char in train_set])
    # Store targets in data_y
    nextword.append(charsdict[test_set])
sentences[1]

Streaming output truncated to the last 5000 lines.
['but', 'no', 'worse', 'than', "what's", 'going', 'on', 'in', 'your', "parent's", 'bedrooms', 'sometimes', 'i', 'want', 'to', 'get', 'on', 'tv', 'and', 'just']
let
['no', 'worse', 'than', "what's", 'going', 'on', 'in', 'your', "parent's", 'bedrooms', 'sometimes', 'i', 'want', 'to', 'get', 'on', 'tv', 'and', 'just', 'let']
loose,
['worse', 'than', "what's", 'going', 'on', 'in', 'your', "parent's", 'bedrooms', 'sometimes', 'i', 'want', 'to', 'get', 'on', 'tv', 'and', 'just', 'let', 'loose,']
but
['than', "what's", 'going', 'on', 'in', 'your', "parent's", 'bedrooms', 'sometimes', 'i', 'want', 'to', 'get', 'on', 'tv', 'and', 'just', 'let', 'loose,', 'but']
can't,
["what's", 'going', 'on', 'in', 'your', "parent's", 'bedrooms', 'sometimes', 'i', 'want', 'to', 'get', 'on', 'tv', 'and', 'just', 'let', 'loose,', 'but', "can't,"]
but
['going', 'on', 'in', 'your', "parent's", 'bedrooms', 'sometimes', 'i', 'want', 'to', 'get', 'on', 'tv', 'and', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['back', 'to', "grandma's", 'house', "it's", 'straight', 'up', 'the', 'road', 'and', 'i', 'was', 'the', 'man', 'of', 'the', 'house,', 'the', 'oldest,', 'so']
my
['to', "grandma's", 'house', "it's", 'straight', 'up', 'the', 'road', 'and', 'i', 'was', 'the', 'man', 'of', 'the', 'house,', 'the', 'oldest,', 'so', 'my']
shoulders
["grandma's", 'house', "it's", 'straight', 'up', 'the', 'road', 'and', 'i', 'was', 'the', 'man', 'of', 'the', 'house,', 'the', 'oldest,', 'so', 'my', 'shoulders']
carried
['house', "it's", 'straight', 'up', 'the', 'road', 'and', 'i', 'was', 'the', 'man', 'of', 'the', 'house,', 'the', 'oldest,', 'so', 'my', 'shoulders', 'carried']
the
["it's", 'straight', 'up', 'the', 'road', 'and', 'i', 'was', 'the', 'man', 'of', 'the', 'house,', 'the', 'oldest,', 'so', 'my', 'shoulders', 'carried', 'the']
weight
['straight', 'up', 'the', 'road', 'and', 'i', 'was', 'the', 'man', 'of', 'the', 'house,', 'the', 'oldest,', 'so', 'my', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['tryna', 'be', 'headstrong', 'but', 'it', 'feels', 'like', 'i', 'slept', 'on', 'my', 'neck', 'wrong', "'cause", "you're", 'moving', 'onto', 'the', 'next,', 'but']
is
['be', 'headstrong', 'but', 'it', 'feels', 'like', 'i', 'slept', 'on', 'my', 'neck', 'wrong', "'cause", "you're", 'moving', 'onto', 'the', 'next,', 'but', 'is']
the
['headstrong', 'but', 'it', 'feels', 'like', 'i', 'slept', 'on', 'my', 'neck', 'wrong', "'cause", "you're", 'moving', 'onto', 'the', 'next,', 'but', 'is', 'the']
respect
['but', 'it', 'feels', 'like', 'i', 'slept', 'on', 'my', 'neck', 'wrong', "'cause", "you're", 'moving', 'onto', 'the', 'next,', 'but', 'is', 'the', 'respect']
gone?
['it', 'feels', 'like', 'i', 'slept', 'on', 'my', 'neck', 'wrong', "'cause", "you're", 'moving', 'onto', 'the', 'next,', 'but', 'is', 'the', 'respect', 'gone?']
'cause
['feels', 'like', 'i', 'slept', 'on', 'my', 'neck', 'wrong', "'cause", "you're", 'moving', 'onto', 'the', 'next,',

[3421,
 7376,
 2967,
 2928,
 2231,
 4779,
 7697,
 471,
 4665,
 6937,
 3402,
 7713,
 2546,
 1175,
 3425,
 4793,
 2955,
 6937,
 2884,
 6857]

In [ ]:
print('Number of sentences in Training set:', len(sentences))

Number of sentences in Training set: 64471


In [ ]:
type(nextword)

list

In [ ]:
num_sent = len(sentences)
num_sent


64471

In [ ]:
from keras.utils import to_categorical
# reshape X to be [samples, time steps, features]
# reshape X to be [samples, time steps, features]
X = np.reshape(sentences, (num_sent, seq_len, 1))
# normalize
X = X / float(len(words))
# one hot encode the output variable
y = to_categorical(nextword)
X.shape
y.shape

(64471, 8548)

In [ ]:
# Length of the vocabulary in words
from keras.layers import Dropout
from keras.optimizers import SGD
vocab_size = len(words)
# The embedding dimension

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='relu'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])




In [ ]:
model.fit(X, y, epochs=2, batch_size=128)


Epoch 1/2
504/504 [==============================] - 18s 33ms/step - loss: 9.4595 - accuracy: 3.9559e-04
Epoch 2/2
504/504 [==============================] - 17s 34ms/step - loss: 8.6872 - accuracy: 1.2557e-04


In [ ]:
import sys
# pick a random seed
start = np.random.randint(0, len(sentences)-1)
pattern = sentences[start]
print("\"", ''.join([intdict[value] for value in pattern]), "\"")
# generate characters
for i in range(20):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(vocab_size)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = intdict[index]
	seq_in = [intdict[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

" onProtoolsSmashthesevocalsanddoaperformanceButweinthevanandheinatourbus "
humblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumblehumble
Done.
